In [47]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import NearestNeighbors
from scipy import stats as st
import matplotlib.pyplot as plt

In [85]:
from sklearn.datasets import fetch_lfw_people
color = True
people = fetch_lfw_people(min_faces_per_person=8, resize=1, color=color)

x = people.data 
y = people.target

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75)

print("Total data points, x pixels, y pixels, color channels")
print(people.images.shape)
print(x_train.shape)

Total data points, x pixels, y pixels, color channels
(4503, 125, 94, 3)
(3377, 35250)


In [85]:
# kNN
k=100
model = NearestNeighbors(n_neighbors=k)
model.fit(x_train,y_train)
test_neighbors = model.kneighbors_graph(x_test)
print("Picking " + str(k) + " nearest neighbors, a total of " + str(x_train.shape[0]) + " training points with " + str(x_train.shape[1]) + " dimensions")

Picking 100 nearest neighbors, a total of 3377 training points with 8742 dimensions


In [86]:
# test_nearest_neighbors looks through test_neighbors and gets the classifications of the k nearest ones
test_nearest_neighbors = np.empty((y_test.shape[0],k))
# y_pred gets the mode of the nearest neighbors
y_pred = np.empty((y_test.shape[0],1))

print("Total number of test points: " + str(x_test.shape[0]))
for i in range(x_test.shape[0]):
    if i%50 == 0:
        print("i=" + str(i))
    test_nearest_neighbors[i] = y_train[np.nonzero(test_neighbors.toarray()[i])]
    y_pred[i] = int(st.mode(test_nearest_neighbors[i], keepdims=True).mode)

print("Done!")

Total number of test points: 1126
i=0
i=50
i=100
i=150
i=200
i=250
i=300
i=350
i=400
i=450
i=500
i=550
i=600
i=650
i=700
i=750
i=800
i=850
i=900
i=950
i=1000
i=1050
i=1100
Done!


In [87]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print("Accuracy: " + str(np.round(100*conf_matrix.trace()/conf_matrix.sum(),2)) + "%")

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Accuracy: 16.7%


In [71]:
# GRAYSCALE
from PIL import Image
import scipy

# Let's pick one image
x_pixels = people.images.shape[1]
y_pixels = people.images.shape[2]
selected_image = np.empty((x_pixels,y_pixels))
num = 255
index = 0
for i in range(x_pixels):
    for j in range(y_pixels):
        selected_image[i][j] = 255*x_train[num][index]
        index += 1

# And print it
image = Image.fromarray(np.array(selected_image, dtype=np.uint8), mode="P")
image.save('original.png')

# Let's use a sharpen kernel
kernel = np.array([[0, -2, 0], 
                    [-2, 9, -2], 
                    [0, -2, 0]])
convolved_image = scipy.signal.convolve2d(selected_image,kernel)

# And print it
image = Image.fromarray(np.array(convolved_image, dtype=np.uint8), mode="P")
image.save('convolved.png')

In [96]:
# COLOR
from PIL import Image
import scipy

# Let's pick one image
x_pixels = people.images.shape[1]
y_pixels = people.images.shape[2]
selected_image = np.empty((x_pixels,y_pixels,3))
selected_image_r = np.empty((x_pixels,y_pixels))
selected_image_g = np.empty((x_pixels,y_pixels))
selected_image_b = np.empty((x_pixels,y_pixels))
num = 255
index = 0
for i in range(x_pixels):
    for j in range(y_pixels):
        selected_image_r[i][j] = 255*x_train[num][index+0]
        selected_image_g[i][j] = 255*x_train[num][index+1]
        selected_image_b[i][j] = 255*x_train[num][index+2]
        selected_image[i][j][0] = 255*x_train[num][index+0]
        selected_image[i][j][1] = 255*x_train[num][index+1]
        selected_image[i][j][2] = 255*x_train[num][index+2]
        index += 3

# And print it
image = Image.fromarray(np.array(selected_image, dtype=np.uint8), mode="RGB")
image.save('original.png')

# Let's use a blur kernel
convolved_image = np.empty((x_pixels,y_pixels,3))
kernel_r = np.array([[0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04]])
kernel_g = np.array([[0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04]])
kernel_b = np.array([[0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04], 
                     [0.04, 0.04, 0.04, 0.04, 0.04]])
convolved_image_r = scipy.signal.convolve2d(selected_image_r,kernel_r)
convolved_image_g = scipy.signal.convolve2d(selected_image_g,kernel_g)
convolved_image_b = scipy.signal.convolve2d(selected_image_b,kernel_b)
for i in range(x_pixels):
    for j in range(y_pixels):
        convolved_image[i][j][0] = convolved_image_r[i][j]
        convolved_image[i][j][1] = convolved_image_g[i][j]
        convolved_image[i][j][2] = convolved_image_b[i][j]

# And print it
image = Image.fromarray(np.array(convolved_image, dtype=np.uint8), mode="RGB")
image.save('convolved.png')